## Declare Factory

In [ ]:
TMVA::Tools::Instance();
// output file with lots of interesting stuff 
auto outputFile = TFile::Open("LowPtEle_ClassificationOutput_patrick.root", "RECREATE");
TMVA::Factory factory("TMVAClassification",
                      outputFile,
                      "!V:ROC:!Silent:Color:!DrawProgressBar:AnalysisType=Classification");

## DataLoader

### Define input variables

In [ ]:
TMVA::DataLoader* loader = new TMVA::DataLoader("dataset");

// add variables here ...
loader->AddVariable("abs_Einv_Pinv");

// these variables do not play a role in the training ...
loader->AddSpectator("pt","pT","GeV",'F');
loader->AddSpectator("eta","eta","",'F');

### Setup input Dataset

In [ ]:
// input ntuple
TString inputFileName = "https://bainbrid.web.cern.ch/bainbrid/misc/tree_numEvent10000_patrick.root";
auto inputFile = TFile::Open( inputFileName );

// signal and background are in the same tree
TTree *sTree = (TTree*)inputFile->Get("lowPtEleIDNtuple/LowPtEle");
TTree *bTree = (TTree*)inputFile->Get("lowPtEleIDNtuple/LowPtEle");

// we have ~5 times more fakes than real electrons in ntuple but weights are applied not here but below
Double_t sWeight = 1.0;
Double_t bWeight = 1.0;
   
// add signal or background trees
loader->AddSignalTree(sTree,sWeight);
loader->AddBackgroundTree(bTree,bWeight);

// weight per electron to give ~same pT/eta distributions 
loader->SetBackgroundWeightExpression("weight");

// these cuts identify which electrons are real or fake 
TCut mycuts = "signal>0.5"; 
TCut mycutb = "signal<0.5"; 

// define split mode for training and testing (currently 50%/50%)
loader->PrepareTrainingAndTestTree(mycuts,
                                   mycutb,
                                   "nTrain_Signal=0:nTrain_Background=0:SplitMode=Random:NormMode=NumEvents:!V" );

# Booking Methods

In [ ]:
// Two "default options" so we can compare 

//Boosted Decision Trees
factory.BookMethod(loader,TMVA::Types::kBDT, "BDT",
                   "!V:NTrees=200:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" );

//Multi-Layer Perceptron (Neural Network)
factory.BookMethod(loader, TMVA::Types::kMLP, "MLP",
                   "!H:!V:NeuronType=tanh:VarTransform=N:NCycles=100:HiddenLayers=N+5:TestRate=5:!UseRegulator" );

## Train all methods

In [ ]:
factory.TrainAllMethods();

## Test  all methods

In [ ]:
factory.TestAllMethods();  

## Evaluate  all methods

In [ ]:
factory.EvaluateAllMethods();  

## Plot ROC Curve

In [ ]:
//%jsroot on
auto c1 = factory.GetROCCurve(loader);
c1->Draw();

## Close outputfile to save all output information (evaluation result of methods)

In [ ]:
outputFile->Close();